In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
#from utils import extract_basic_features

#import wfdb
import os
#import wfdb.processing as wp
import matplotlib.pyplot as plt
from scipy import signal
#from utils import find_noise_features, extract_basic_features
import shutil
import gc
import time
import random as rn
#from lightgbm import LGBMClassifier
from scipy import sparse
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold,StratifiedKFold
#from xgboost import XGBClassifier
import pywt
import warnings
import scipy.io as sio

#from resnet_ecg.utils import one_hot,get_batches
from resnet_ecg.ecg_preprocess import ecg_preprocessing


path = '/media/jdcloud/'

warnings.filterwarnings("ignore")

In [2]:
def wavelet(ecg,wavefunc,lv,m,n):   #
    
    coeff = pywt.wavedec(ecg,wavefunc,mode='sym',level=lv)   #
    #sgn = lambda x: 1 if x > 0 else -1 if x < 0 else 0

    for i in range(m,n+1):  
        cD = coeff[i]
        for j in range(len(cD)):
            Tr = np.sqrt(2*np.log(len(cD)))  
            if cD[j] >= Tr:
                coeff[i][j] = np.sign(cD[j]) - Tr 
            else:
                coeff[i][j] = 0   
                
    denoised_ecg = pywt.waverec(coeff,wavefunc)
    return denoised_ecg

In [3]:
def wavelet_db6(sig):
    """
    R J, Acharya U R, Min L C. ECG beat classification using PCA, LDA, ICA and discrete
     wavelet transform[J].Biomedical Signal Processing and Control, 2013, 8(5): 437-448.

    param sig: 1-D numpy Array
    return: 1-D numpy Array
    """

    coeffs = pywt.wavedec(sig, 'db6', level=9)

    coeffs[-1] = np.zeros(len(coeffs[-1]))

    coeffs[-2] = np.zeros(len(coeffs[-2]))

    coeffs[0] = np.zeros(len(coeffs[0]))

    sig_filt = pywt.waverec(coeffs, 'db6')

    return sig_filt

In [4]:
path = "/media/uuser/data/ysecgtest/training_data_pre/"
data = np.load(path + "TRAIN0001.npy")

In [5]:
for i,v in enumerate(data[:,0]):
    if v != 0 :
        print(v)
        print(i)
        break

-0.39426738
20736


In [6]:
labels = pd.read_csv("/media/uuser/data/final_codes/final_run_semi/reference.csv")
raw_IDs = labels["File_name"].values.tolist()
raw_IDs[-10:]

['TRAIN6491',
 'TRAIN6492',
 'TRAIN6493',
 'TRAIN6494',
 'TRAIN6495',
 'TRAIN6496',
 'TRAIN6497',
 'TRAIN6498',
 'TRAIN6499',
 'TRAIN6500']

In [7]:
labels.iloc[5320]

File_name    TRAIN5321
label1               6
label2             NaN
label3             NaN
label4             NaN
label5             NaN
label6             NaN
label7             NaN
label8             NaN
Name: 5320, dtype: object

In [13]:
def read_data_seg_back(data_path, split="Train", preprocess=False, fs=500, newFs=256, winSecond=10, winNum=10, n_index=0):
    """ Read data """

    # Fixed params
    # n_index = 0
    n_class = 9
    winSize = winSecond * fs
    new_winSize = winSecond * newFs
    # Paths
    path_signals = os.path.join(data_path, split)
    
    labels = pd.read_csv("/media/uuser/data/final_codes/final_run_semi/reference.csv")
    raw_IDs = labels["File_name"].values.tolist()

    # Read time-series data
    channel_files = raw_IDs#os.listdir(path_signals)
    # print(channel_files)
    channel_files.sort()
    n_channels = 12  # len(channel_files)
    # posix = len(split) + 5

    # Initiate array
    list_of_channels = []
    
    path_signals = "/media/uuser/data/ysecgtest/training_data_pre/"
    path  = "/media/uuser/data/ysecgtest/training_data/"
    
    i_ch = 0

    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1', 'II', 'aVR', 'V3', 'III', 'V5']

    for i_ch, fil_ch in tqdm(enumerate(channel_files[:])):  # tqdm
        
        if i_ch % 1000 == 0:
            print(i_ch)
            
        
        ecg = np.load(os.path.join(path_signals, fil_ch)+"_db4.npy")  
        
        for i,v in enumerate(ecg[:,0]):
            if v != 0 :
                ecg_length = 23296 - i
                break
        ecg = ecg[i:,:]
        #print(ecg.shape)
        #print(ecg_length)
        '''
        if ecg_length > fs * winNum * winSecond:
            print(" too long !!!", ecg_length)
            ecg_length = fs * winNum * winSecond
        if ecg_length < 4500:
            print(" too short !!!", ecg_length)
            break
        ''' 
        slide_steps = int((ecg_length - new_winSize) / (winNum-1))#int((ecg_length - winSize) / winSecond)

        if ecg_length <= 2560:
            slide_steps = 0
            
        X = np.zeros((winNum, new_winSize, n_channels)).astype('float32') 
        #X_db4 = np.zeros((winNum, new_winSize, n_channels)).astype('float32') 
        #X_db6 = np.zeros((winNum, new_winSize, n_channels)).astype('float32') 
        
        for ind in range(winNum):
            
            n_index = ind
            
            ecg_channels = np.zeros((new_winSize, n_channels)).astype('float32') 
            #ecg_channels_db4 = np.zeros((new_winSize, n_channels)).astype('float32') 
            #ecg_channels_db6 = np.zeros((new_winSize, n_channels)).astype('float32') 
            
            for i_n, ch_name in enumerate(channel_name):
                #print(ecg[i_n])
                if ecg_length <= 2560:
                    ecg_channels[:ecg_length, i_n] = ecg[n_index * slide_steps:,i_n].T
                else:
                    ecg_channels[:, i_n] = ecg[n_index * slide_steps:n_index * slide_steps + new_winSize,i_n].T

            X[ind, :, :] = ecg_channels
            #X_db4[ind, :, :] = ecg_channels_db4
            #X_db6[ind, :, :] = ecg_channels_db6
        np.save(path + "{}_db4.npy".format(fil_ch),X)
        #np.save("training_data/{}.npy".format(fil_ch.strip(".mat")),X)
        #np.save("training_data/{}_db4.npy".format(fil_ch.strip(".mat")),X_db4)
        #np.save("training_data/{}_db6.npy".format(fil_ch.strip(".mat")),X_db6)
                
    return "OK!!!"

In [14]:
read_data_seg_back(data_path="")

2it [00:00, 15.22it/s]

0


1002it [01:05, 14.78it/s]

1000


2002it [02:10, 15.54it/s]

...

5000


6002it [06:19, 14.73it/s]

6000


6500it [06:51, 15.11it/s]


'OK!!!'

In [4]:
def read_data_seg(data_path, split="Train", preprocess=False, fs=500, newFs=256, winSecond=10, winNum=10, n_index=0):
    """ Read data """

    # Fixed params
    # n_index = 0
    n_class = 9
    winSize = winSecond * fs
    new_winSize = winSecond * newFs
    # Paths
    path_signals = os.path.join(data_path, split)

    # Read labels82975 and one-hot encode
    # label_path = os.path.join(data_path, "reference.txt")
    # labels = pd.read_csv(label_path, sep='\t',header = None)
    # labels = pd.read_csv("reference.csv")

    # Read time-series data
    channel_files = os.listdir(path_signals)
    # print(channel_files)
    channel_files.sort()
    n_channels = 12  # len(channel_files)
    # posix = len(split) + 5

    # Initiate array
    list_of_channels = []

    
    i_ch = 0

    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1', 'II', 'aVR', 'V3', 'III', 'V5']

    for i_ch, fil_ch in tqdm(enumerate(channel_files[:])):  # tqdm
        
        if i_ch % 1000 == 0:
            print(i_ch)
            
        ecg = sio.loadmat(os.path.join(path_signals, fil_ch))
        ecg_length = ecg["I"].shape[1]

        if ecg_length > fs * winNum * winSecond:
            print(" too long !!!", ecg_length)
            ecg_length = fs * winNum * winSecond
        if ecg_length < 4500:
            print(" too short !!!", ecg_length)
            break

        slide_steps = int((ecg_length - winSize) / (winNum-1))#int((ecg_length - winSize) / winSecond)

        if ecg_length <= 4500:
            slide_steps = 0
            
        X = np.zeros((winNum, new_winSize, n_channels)).astype('float32') 
        #X_db4 = np.zeros((winNum, new_winSize, n_channels)).astype('float32') 
        #X_db6 = np.zeros((winNum, new_winSize, n_channels)).astype('float32') 
        
        for ind in range(winNum):
            
            n_index = ind
            
            ecg_channels = np.zeros((new_winSize, n_channels)).astype('float32') 
            #ecg_channels_db4 = np.zeros((new_winSize, n_channels)).astype('float32') 
            #ecg_channels_db6 = np.zeros((new_winSize, n_channels)).astype('float32') 
            
            for i_n, ch_name in enumerate(channel_name):
                ecg_channels[:, i_n] = signal.resample(ecg[ch_name]
                                                       [:, n_index * slide_steps:n_index * slide_steps + winSize].T
                                                       , new_winSize).T
                if preprocess:
                    print("preprocess")
                    data = ecg_preprocessing(ecg_channels[:, i_n].reshape(1, new_winSize), 'sym8', 8, 3, newFs,
                                            removebaseline=False,normalize=False)
                    #ecg_channels_db4[:,i_n] = wavelet(ecg_channels[:,i_n],'db4',4,2,4)
                    ecg_channels[:, i_n] = data[0]
                    
                    #ecg_channels_db6[:,i_n] = wavelet_db6(ecg_channels[:,i_n])
                    #print(ecg_channels_db6[:, i_n])
                else:
                    pass
                    #ecg_channels[:, i_n] = ecg_channels[:, i_n]
                    
                
            X[ind, :, :] = ecg_channels
            #X_db4[ind, :, :] = ecg_channels_db4
            #X_db6[ind, :, :] = ecg_channels_db6
        np.save("training_data/{}_ori.npy".format(fil_ch.strip(".mat")),X)
        #np.save("training_data/{}.npy".format(fil_ch.strip(".mat")),X)
        #np.save("training_data/{}_db4.npy".format(fil_ch.strip(".mat")),X_db4)
        #np.save("training_data/{}_db6.npy".format(fil_ch.strip(".mat")),X_db6)
        
        ''' 
        X = np.zeros((winNum, new_winSize, n_channels)).astype('float32') 
        for ind in range(winNum):
            ecg_channels = np.zeros((new_winSize, n_channels)).astype('float32') 
            for i_n, ch_name in enumerate(channel_name):
                ecg_channels[:, i_n] = signal.resample(ecg[ch_name]
                                                       [:, n_index * slide_steps:n_index * slide_steps + winSize].T
                                                       , new_winSize).T
                ecg_channels[:,i_n] = wavelet(ecg_channels[:,i_n],'db4',4,2,4)[0]
                
            X[ind, :, :] = ecg_channels
            
        '''
        
    return "OK!!!"

In [5]:
data = read_data_seg(path,preprocess=False)

4it [00:00, 33.31it/s]

0


1006it [00:26, 37.86it/s]

1000


2007it [00:53, 36.53it/s]

...

5007it [02:12, 36.81it/s]

5000


6004it [02:39, 33.70it/s]

6000


6689it [03:00, 37.09it/s]


In [7]:
a = os.listdir("training_data/")

In [9]:
a.sort()

In [15]:
len(a)/2

6500.0

In [7]:
data.shape

(10, 2560, 12)

In [ ]:
def read_data_labels_enhance(data_path, split = "Train",preprocess=True):
    """ Read data """

    # Fixed params
    n_class = 2
    n_steps = 3000#2560

    # Paths
    path_signals = os.path.join(data_path, split)

    # Read labels and one-hot encode
    labels = pd.read_csv("reference.csv")
    #2156+224+672+654+180+826+534+504+1953
    
    # Read time-series data
    channel_files = os.listdir(path_signals)
    #print(channel_files)
    channel_files.sort()
    n_channels = 12#len(channel_files)
    #posix = len(split) + 5

    # Initiate array
    list_of_channels = []
    X = np.zeros((len(channel_files), n_steps, n_channels))
    i_ch = 0
    
    data_x = []
    data_y = []
    
    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1','II', 'aVR', 'V3', 'III', 'V5']
    channel_mid_name = ['II','aVR','V2','V5']
    channel_post_name = ['III','aVF','V3','V6']
    
    for i_ch,fil_ch in tqdm(enumerate(channel_files[:1])):
        #print(fil_ch)
        labels_list = labels.iloc[i_ch].values[:]#.dropna()
        #print(labels_list)
        ecg = sio.loadmat(os.path.join(path_signals,fil_ch))
        
        if True:#7 in labels_list[1:] or 4 in labels_list[1:]:
            for i_filter in range(3):
                data_y.append(list(labels_list))
                
                ecg_channels = np.zeros((n_steps, n_channels))

                for i_n,ch_name in enumerate(channel_name[:]):

                    # method 1
                    '''  '''
                    if ch_name in channel_mid_name:
                        mid_ind = int(ecg[ch_name].T.shape[0]/2)
                        ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[mid_ind-2500:mid_ind+2500],n_steps).T 
                    elif ch_name in channel_post_name:
                        ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[-5000:],n_steps).T
                    else:
                        ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[:5000],n_steps).T
                    
                    #method 2
                    #ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T,n_steps).T
                    
                    #print(ecg_channels[:,i_n].shape)
                    if preprocess and i_filter == 0:
                        data = ecg_preprocessing(ecg_channels[:,i_n].reshape(1,n_steps),'sym8',8,3,n_steps/10)
                        ecg_channels[:,i_n] = data[0]#ecg['data']
                    elif i_filter == 1:
                        ecg_channels[:,i_n] = wavelet(ecg_channels[:,i_n],'db4',4,2,4)[0]
                    elif i_filter == 1:
                        ecg_channels[:,i_n] = wavelet_db6(ecg_channels[:,i_n].reshape(1,n_steps))[0]
                    else:
                        pass
                        #ecg_channels[:,i_n] = ecg_channels[:,i_n]
                X[i_ch,:,:] = ecg_channels
                data_x.append(ecg_channels)
        else:
                data_y.append(list(labels_list))
                ecg_channels = np.zeros((n_steps, n_channels))
                for i_n,ch_name in enumerate(channel_name[:]):

                    # method 1
                    '''
                    if ch_name in channel_mid_name:
                        mid_ind = int(ecg[ch_name].T.shape[0]/2)
                        ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[mid_ind-2500:mid_ind+2500],2560).T 
                    elif ch_name in channel_post_name:
                        ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[-5000:],2560).T
                    else:
                        ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T[:5000],2560).T
                    '''
                    #method 2
                    ecg_channels[:,i_n] = signal.resample(ecg[ch_name].T,n_steps).T

                    if preprocess:
                        data = ecg_preprocessing(ecg_channels[:,i_n].reshape(1,n_steps),'sym8',8,3,n_steps/10)
                        ecg_channels[:,i_n] = data[0]#ecg['data']
                    else:
                        pass

                X[i_ch,:,:] = ecg_channels
                data_x.append(ecg_channels)

    # Return 
    return np.array(data_x).astype('float32'),data_y#X